In [2]:
!pip install pymysql

    100% |████████████████████████████████| 81kB 355kB/s ta 0:00:01


In [1]:
import pandas as pd
import math 
import pymysql 
import sys 
import time 
import sqlite3
import csv
import pprint
import re



In [2]:
#generate 1.model transform dictionary 2.regular expression dictionary
def gen_transform_dict(csvfile):    
    brand_list=[]
    modelTransform={}
    with open(csvfile, "r", encoding='Big5') as f:
        reader = csv.DictReader(f)
        for line in reader:
            brand=line['brand'].lower()
            key=line['key'].strip().lower()
            value=line['value'].strip().lower()
            if brand not in modelTransform.keys():
                brand_list.append(brand)
                modelTransform[brand]={key:value}
            else:
                modelTransform[brand].update({key:value})
    regex_strs={}
    for brand in brand_list:
        regex_str=""
        for keys in modelTransform[brand].keys():
            regex_str+='{}|'.format(keys)
            regex_strs[brand]=regex_str.strip('|') 
            
    print(brand_list)
    return (modelTransform,regex_strs)


# m is model transform dict, r is regular expression dict
m,r=gen_transform_dict('./yahoo/yahoo-model-transformation.csv')

['mitsubishi', 'nissan', 'toyota', 'volkswagen', 'ford', 'honda', 'lexus', 'mazda']


In [16]:
m

{'ford': {'econovan': 'econovan',
  'ecosport': 'ecosport',
  'escape': 'escape',
  'fiesta': 'fiesta',
  'fiesta 4d': 'fiesta',
  'fiesta 5d': 'fiesta',
  'fiesta-4d': 'fiesta',
  'fiesta-5d': 'fiesta',
  'fiesta4d': 'fiesta',
  'fiesta5d': 'fiesta',
  'focus': 'focus',
  'focus 4d': 'focus',
  'focus 5d': 'focus',
  'focus powershift': 'focus powershift',
  'focus-4d': 'focus',
  'focus-5d': 'focus',
  'focus4d': 'focus',
  'focus5d': 'focus',
  'i': 'i-max',
  'i max': 'i-max',
  'i-max': 'i-max',
  'imax': 'i-max',
  'kuga': 'kuga',
  'mondeo': 'mondeo',
  'mondeo tdci': 'mondeo tdci',
  'mustang': 'mustang',
  'ranger': 'ranger',
  'tourneo': 'tourneo custom',
  'tourneo custom': 'tourneo custom',
  '載卡多': 'econovan'},
 'honda': {'accord': 'accord',
  'city': 'city',
  'civic': 'civic',
  'cr': 'cr-v',
  'cr v': 'cr-v',
  'cr z': 'cr-z',
  'cr-v': 'cr-v',
  'cr-z': 'cr-z',
  'crv': 'cr-v',
  'crz': 'cr-z',
  'fit': 'fit',
  'hr v': 'hr-v',
  'hr-v': 'hr-v',
  'hrv': 'hr-v',
  'ins

In [3]:
r

{'ford': 'focus powershift|tourneo custom|mondeo tdci|fiesta 4d|fiesta 5d|fiesta-4d|fiesta-5d|fiesta4d|fiesta5d|focus 4d|focus 5d|focus-4d|focus-5d|focus4d|focus5d|mustang|econovan|ecosport|ranger|focus|tourneo|escape|fiesta|mondeo|i max|i-max|imax|kuga|載卡多|i',
 'honda': 'insight hybrid|insight|odyssey|accord|legend|civic|nsx|city|cr-v|cr-z|hr-v|cr v|cr z|hr v|crv|crz|hrv|fit|cr',
 'lexus': 'ct|es|gs|is|isseries|is series|is-series|isc|lc|lfa|ls|lx|nx|rc|rx|sc',
 'mazda': '2|3 4d|3 5d|3-4d|3-5d|3|5|6|6 wagon|cx-3|cx-5|cx-7|cx-9|mx-5|tribute|cx 3|cx 5|cx 7|cx 9|mx 5|cx3|cx5|cx7|cx9|mx5',
 'mitsubishi': 'lancer sportback|lancer-sportback|lancersportback|savrin inspire|savrin-inspire|savrininspire|lancer fortis|lancer-fortis|fortis|super zinger|super-zinger|lancerfortis|superzinger|boss zinger|boss-zinger|bosszinger|pajero 三門款|pajero-三門款|pajero 五門款|pajero-五門款|space gear|space-gear|colt plus|colt-plus|lancer io|lancer-io|io|outlander|pajero三門款|pajero五門款|spacegear|coltplus|lancerio|grunder|

In [12]:
class CK_mySQL: 
    IP = '192.168.196.36'
    user='team1'
    passwd = 'team1'
    db= 'team1'
    table='crawl_temp'

def model_transform(model_dict, regex_dict, brand, model):
    regex=regex_dict[brand]
    model_list = re.findall('{}'.format(regex),model,re.IGNORECASE)
    model_list.sort(key=len, reverse=True)
    new_model=model_dict[brand][model_list[0].lower()]
    return new_model


def gen_newdata(CK_mySQL):
    conn1 = pymysql.connect(host=CK_mySQL.IP, port=3306, user=CK_mySQL.user, passwd=CK_mySQL.passwd, db=CK_mySQL.db,charset='utf8') 
    cur1 = conn1.cursor() 
    
    sql="SELECT * FROM {} where years>=2008;".format(CK_mySQL.table)
    cur1.execute(sql)

    newdata=[]
    for row in cur1:

        try:
            url=row[1]
            brand= row[3].lower()
            model= row[2]+row[4]
            # model_transfrom is used here
            new=model_transform(m,r,brand,model) 
            if (new != row[4]):
                newdata.append((new,url))

        except Exception as e:
            print(brand,model,url)
            with open('fail_model.csv','a') as f:
                f.write('{},{},{},{}\n'.format(brand, row[4], row[2], url))

    conn1.close()        
    return newdata
newdata=gen_newdata(CK_mySQL)

mitsubishi MITSUBISHI NEW VERYCA <8??>??????!NEW VERYCA http://www.sum.com.tw/carinfo.php?id=253181
nissan NISSAN ALL NEW CABSTAR 2008/2 ??.??? 12?.FRP?ALL NEW CABSTAR http://www.sum.com.tw/carinfo.php?id=280382
mitsubishi MITSUBISHI VERYCA ?????VERYCA http://www.sum.com.tw/carinfo.php?id=288203
mitsubishi MITSUBISHI ALL NEW DELICA ??????ALL NEW DELICA http://www.sum.com.tw/carinfo.php?id=297313
benz BENZ E-CLASS W212  BENZ E350 ??? ???? ????E http://www.sum.com.tw/carinfo.php?id=315799
benz BENZ E-CLASS W212  ?2014?? BENZ E200 ???? ???E http://www.sum.com.tw/carinfo.php?id=315804
benz BENZ S-CLASS W221  2011?  ?  S350  3.5  S http://www.sum.com.tw/carinfo.php?id=315805
benz BENZ M-CLASS W166 2012?  ?  ML350  3.5  ???? M http://www.sum.com.tw/carinfo.php?id=315810
benz BENZ GLK 350 2011?  ?  GLK350  3.5 GLK 350 http://www.sum.com.tw/carinfo.php?id=315812
bmw BMW 3 SERIES SEDAN F30  ?2012?? BENZ 328I 2.03 SERIES SEDAN F30 http://www.sum.com.tw/carinfo.php?id=315814
bmw BMW 5 SERIES SEDA

mitsubishi MITSUBISHI DELICA?? ??YES???????????????????/DELICA貨車 http://www.sum.com.tw/carinfo.php?id=395933
benz BENZ C-CLASS SEDAN W204  BENZ C250 ??1.8cc ??C http://www.sum.com.tw/carinfo.php?id=395959
benz BENZ C-CLASS ESTATE  2011年賓士 C200 1800cc 5門旅行車 優質車C http://www.sum.com.tw/carinfo.php?id=395978
benz BENZ C-CLASS SEDAN W204  BENZ C-CLASS SEDAN WC http://www.sum.com.tw/carinfo.php?id=396071
benz BENZ C-CLASS SEDAN W204  BENZ C250 ??? ??1.8cC http://www.sum.com.tw/carinfo.php?id=396115
mitsubishi MITSUBISHI CANTER  ??YES???????????????/????/CANTER http://www.sum.com.tw/carinfo.php?id=396144
benz BENZ S-CLASS W221  S350 ??3? ?????? ?????S http://www.sum.com.tw/carinfo.php?id=396154
mitsubishi MITSUBISHI VERYCA?? 2???????/???/????/???VERYCA廂車 http://www.sum.com.tw/carinfo.php?id=396181
volvo VOLVO S60 VOLVO S60S60 http://www.sum.com.tw/carinfo.php?id=396263
benz BENZ C-CLASS SEDAN W205  BENZ C300 ??? ???? ?C http://www.sum.com.tw/carinfo.php?id=396290
bmw BMW 5 SERIES SEDAN F10 ??

subaru SUBARU FORESTER  4WD/ABS/電動天窗/可全貸/附發票FORESTER http://www.sum.com.tw/carinfo.php?id=407517
benz BENZ C-CLASS SEDAN W205  15?C300AMG P1?? ???1C http://www.sum.com.tw/carinfo.php?id=407581
benz BENZ E-CLASS W212  1.8 ???? ??? ??????E http://www.sum.com.tw/carinfo.php?id=407602
mitsubishi MITSUBISHI VERYCA?? 1.3 ?? ??/?????/?????VERYCA廂車 http://www.sum.com.tw/carinfo.php?id=407603
bmw BMW X1 E84  ??????4??????20D?????????????????X1 E84 http://www.sum.com.tw/carinfo.php?id=407621
mitsubishi MITSUBISHI DELICA??  ????????,????,8????,????DELICA廂車 http://www.sum.com.tw/carinfo.php?id=407629
suzuki SUZUKI SUPER CARRY 2008年 吉利【1.6L/大馬力省油輕鬆載/外型搶眼內裝舒適】可分期/可開發票SUPER CARRY http://www.sum.com.tw/carinfo.php?id=407642
volvo VOLVO S60 特價!T5一手只跑4.5萬市面少有全車原鈑件如新，免頭款要買要快!S60 http://www.sum.com.tw/carinfo.php?id=407675
mitsubishi MITSUBISHI VERYCA?? ??!??????????????????????VERYCA廂車 http://www.sum.com.tw/carinfo.php?id=407676
subaru SUBARU FORESTER  4WD/ABS/6安/里程保證/可全貸/附發票FORESTER http://www.sum.com.t

porsche PORSCHE CAYENNE  稀車釋出＊一手．車身美．車況佳CAYENNE http://www.sum.com.tw/carinfo.php?id=412605
mitsubishi MITSUBISHI VERYCA ?? ??!16??????1????????????VERYCA 貨車 http://www.sum.com.tw/carinfo.php?id=412606
suzuki SUZUKI SUPER CARRY SUZUKI SUPER CARRYSUPER CARRY http://www.sum.com.tw/carinfo.php?id=412629
benz BENZ C-CLASS W204  SUM?????????C300 AMG ?? ??C http://www.sum.com.tw/carinfo.php?id=412644
benz BENZ E-CLASS W212  2014?? E200 AVANTGARDE ???E http://www.sum.com.tw/carinfo.php?id=412694
benz BENZ E-CLASS W212  ?? ?? 220CDI ?? ??!! E http://www.sum.com.tw/carinfo.php?id=412704
bmw BMW 1 SERIES F20 ??2?2 2016 ??? 118i ??? ?? L1 SERIES F20 http://www.sum.com.tw/carinfo.php?id=412724
bmw BMW 2 SERIES ACTIVE TOURER  BMW220 2015??????2 SERIES ACTIVE TOURER http://www.sum.com.tw/carinfo.php?id=412730
mitsubishi MITSUBISHI DELICA?? ???~??7000??~????~9??~??DELICA貨車 http://www.sum.com.tw/carinfo.php?id=412788
mitsubishi MITSUBISHI DELICA??  ?????~????~??+??~???~DELICA貨車 http://www.sum.com.tw/c

nissan NISSAN CABSTAR  2008? ?? ??????16?????CABSTAR http://www.sum.com.tw/carinfo.php?id=415651
mitsubishi MITSUBISHI DELICA??  2009? ???2.4 ??130000???DELICA貨車 http://www.sum.com.tw/carinfo.php?id=415652
subaru SUBARU FORESTER  14年Forester XT/4WD/水平對臥/渦輪增壓/無段變速/保固一年FORESTER http://www.sum.com.tw/carinfo.php?id=415654
mitsubishi MITSUBISHI LEADCA  特價!一手14呎貨蓬只跑2.2萬全車無碰撞原廠保養，附發票免頭款不用殺價就很便宜LEADCA http://www.sum.com.tw/carinfo.php?id=415667
benz BENZ C-CLASS COUPE W204 BENZ C-CLASS COUPE W204C http://www.sum.com.tw/carinfo.php?id=415674
benz BENZ C-CLASS SEDAN W204  BENZ C-CLASS SEDAN W204 C http://www.sum.com.tw/carinfo.php?id=415682
bmw BMW 1 SERIES HATCHBACK 5D F20 2015年式 BMW 118i M版 M版大包、鋁圈 可全貸免頭款 1 SERIES HATCHBACK 5D F20 http://www.sum.com.tw/carinfo.php?id=415693
bmw BMW X3 F25  X3  柴油2.0 原廠保養 汎德總代理 全景天窗X3 F25 http://www.sum.com.tw/carinfo.php?id=415700
vw VW GOLF VI VW GOLF VIGOLF VI http://www.sum.com.tw/carinfo.php?id=415713
mitsubishi MITSUBISHI DELICA 廂車 特價!一手八人座只跑1.5萬全車無碰撞根本新

...

mitsubishi MitsubishiDelicadelica https://usedcar.u-car.com.tw/objectdetail.aspx?usedcarid=71df2a6c4c03465ea4464d8f0bb158cf
bmw BMW7 Series7 series https://usedcar.u-car.com.tw/objectdetail.aspx?usedcarid=720f6a78f3724b8f8f73d3664bb9ef8e
bmw BMW1 Series1 series https://usedcar.u-car.com.tw/objectdetail.aspx?usedcarid=7217fedbd705450d9987d1c417b61d18
benz Mercedes-BenzC-Classc-class https://usedcar.u-car.com.tw/objectdetail.aspx?usedcarid=722f90ca4ef743e2be5c17bf0861e890
bmw BMW3 Series3 series https://usedcar.u-car.com.tw/objectdetail.aspx?usedcarid=723dd4a260924b7fa0fd654eda9c3d55
benz Mercedes-BenzE-Classe-class https://usedcar.u-car.com.tw/objectdetail.aspx?usedcarid=723f990d89df449090fbb2202cfdb474
benz Mercedes-BenzA-Classa-class https://usedcar.u-car.com.tw/objectdetail.aspx?usedcarid=72632ae73ab748c2b74013d2230dc432
bmw BMW5 Series5 series https://usedcar.u-car.com.tw/objectdetail.aspx?usedcarid=72996aa8947b43f5836e5c8715b22d05
bmw BMW3 Series3 series https://usedcar.u-car.com.t

benz Mercedes-BenzC-Classc-class https://usedcar.u-car.com.tw/objectdetail.aspx?usedcarid=9ea4a1a672744d9ea842028b0038ecbd
bmw BMW1 Series1 series https://usedcar.u-car.com.tw/objectdetail.aspx?usedcarid=9eaf43c02fcc4a2ea4f93d1dcbabbbbe
bmw BMW5 Series5 series https://usedcar.u-car.com.tw/objectdetail.aspx?usedcarid=9ed4e56fa7ec4ff3beb7e9db1ce3cc80
benz Mercedes-BenzE-Classe-class https://usedcar.u-car.com.tw/objectdetail.aspx?usedcarid=9f3a30cc424948528529e159cdb7a3b5
benz Mercedes-BenzC-Classc-class https://usedcar.u-car.com.tw/objectdetail.aspx?usedcarid=9f550c995b8344ecad5282cf4e034a2c
benz Mercedes-BenzC-Classc-class https://usedcar.u-car.com.tw/objectdetail.aspx?usedcarid=9f5adba06b2c47349cbde7f8b0b7d75e
benz Mercedes-BenzGT-Classgt-class https://usedcar.u-car.com.tw/objectdetail.aspx?usedcarid=9f6c589d42394247a011984d8db6f658
bmw BMWX6x6 https://usedcar.u-car.com.tw/objectdetail.aspx?usedcarid=9f8f9ae8f7bd49c2982056a99a4b434b
benz Mercedes-BenzCLS-Classcls-class https://usedcar.

benz Mercedes-BenzS-Classs-class https://usedcar.u-car.com.tw/objectdetail.aspx?usedcarid=d0124c1dc5094da4ac7a15e02eaa7773
bmw BMW1 Series1 series https://usedcar.u-car.com.tw/objectdetail.aspx?usedcarid=d02f006e7af24d3496da5c8756c9a824
benz Mercedes-BenzC-Classc-class https://usedcar.u-car.com.tw/objectdetail.aspx?usedcarid=d0425f245d3a4e6cb1644a0540d02c6f
benz Mercedes-BenzC-Classc-class https://usedcar.u-car.com.tw/objectdetail.aspx?usedcarid=d04ea82a16cf4808a7360f05e7457e0f
bmw BMWX5x5 https://usedcar.u-car.com.tw/objectdetail.aspx?usedcarid=d05285dc5d084aaeaefadf029e953199
benz Mercedes-BenzCLS-Classcls-class https://usedcar.u-car.com.tw/objectdetail.aspx?usedcarid=d0b0aab7b1ca46d9815400cb652951a7
benz Mercedes-BenzS-Classs-class https://usedcar.u-car.com.tw/objectdetail.aspx?usedcarid=d13bc637b2404103a7791c3d0dd6947b
benz Mercedes-BenzGLA-Classgla-class https://usedcar.u-car.com.tw/objectdetail.aspx?usedcarid=d13f4a3070d247008418e0afaa5ffd80
mitsubishi MitsubishiCantercanter http

suzuki SuzukiSwiftswift https://usedcar.u-car.com.tw/objectdetail.aspx?usedcarid=fcf8b94a6cc84448b377c6e718ea59b6
bmw BMWX5x5 https://usedcar.u-car.com.tw/objectdetail.aspx?usedcarid=fd008df754d84e0fac578d22e3393b90
bmw BMW3 Series3 series https://usedcar.u-car.com.tw/objectdetail.aspx?usedcarid=fd707d63838f4a949d3329b291034fe9
benz Mercedes-BenzE-Classe-class https://usedcar.u-car.com.tw/objectdetail.aspx?usedcarid=fd7b5d4a4ef54f749fd32786615c5126
benz Mercedes-BenzC-Classc-class https://usedcar.u-car.com.tw/objectdetail.aspx?usedcarid=fd7ebb4553cc44528baae77c482f9106
bmw BMW5 Series5 series https://usedcar.u-car.com.tw/objectdetail.aspx?usedcarid=fdab21998a6a4a26822383335078c78a
benz Mercedes-BenzCLA-Classcla-class https://usedcar.u-car.com.tw/objectdetail.aspx?usedcarid=fddf318a6c1e461f89f554b134873b0d
benz Mercedes-BenzE-Classe-class https://usedcar.u-car.com.tw/objectdetail.aspx?usedcarid=fdeff035bee144eeb0b37d75d271c944
benz Mercedes-BenzCLA-Classcla-class https://usedcar.u-car.c

benz M-BENZ-CLA-CLASS CLA 250CLA-CLASS CLA 250 https://www.abccar.com.tw/buy-car/738248
bmw BMW-3 SERIES 335i3 SERIES 335i https://www.abccar.com.tw/buy-car/738338
mitsubishi MITSUBISHI-DELICA 2.4DELICA 2.4 https://www.abccar.com.tw/buy-car/738368
benz M-BENZ-R-CLASS R 350R-CLASS R 350 https://www.abccar.com.tw/buy-car/738399
mitsubishi MITSUBISHI-DELICA 2.4DELICA 2.4 https://www.abccar.com.tw/buy-car/738717
bmw BMW-X1 SERIES 23dX1 SERIES 23d https://www.abccar.com.tw/buy-car/738812
benz M-BENZ-C-CLASS C 250C-CLASS C 250 https://www.abccar.com.tw/buy-car/738827
benz M-BENZ-E-CLASS E 200E-CLASS E 200 https://www.abccar.com.tw/buy-car/738831
benz M-BENZ-E-CLASS E 63 AMGE-CLASS E 63 AMG https://www.abccar.com.tw/buy-car/738932
benz M-BENZ-C-CLASS C 250C-CLASS C 250 https://www.abccar.com.tw/buy-car/739152
bmw BMW-5 SERIES 528i5 SERIES 528i https://www.abccar.com.tw/buy-car/739162
benz M-BENZ-E-CLASS E 200E-CLASS E 200 https://www.abccar.com.tw/buy-car/739444
bmw BMW-1 SERIES 118i1 SERIES 

In [13]:
len(newdata)

2957

In [15]:
import sys
conn2 = pymysql.connect(host=CK_mySQL.IP, port=3306, user=CK_mySQL.user, passwd=CK_mySQL.passwd, db=CK_mySQL.db,charset='utf8') 
cur2 = conn2.cursor()
l=len(newdata)
count=0
for idx, car in enumerate(newdata):
    newmodel=car[0]
    url=car[1]
    try: 
        sql='UPDATE {} SET model=%s WHERE url= %s;'.format(CK_mySQL.table)
        cur2.execute(sql,(newmodel, url) )  
        
        if idx % 500 == 0:
            conn2.commit()

    except Exception as e: 
        print(e) 
        conn2.rollback()
    count+=1
    printStr='now is {}, {}% finished'.format(count, round(count/l*100)) 
    sys.stdout.write('\r' + printStr) 
conn2.commit()
conn2.close()

    


now is 2957, 100% finished

In [ ]:
# years= row[11]
#     gasoline=row[16]
#     cc=row[7]
#     wd na
#     doors=row[5]
#     transmission=row[8]
#     equip=row[9]
    